# Import Packages

In [1]:
import pickle
import nltk
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import spacy
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
from tqdm import tqdm
from gensim.models.phrases import Phrases
# from spacy.matcher import PhraseMatcher

2023-08-11 10:28:35.250701: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
pd.set_option('display.max_rows', None)

# Load Data

In [2]:
df = pd.read_csv("/Users/irene/CloudStation/Courses/ARP/ARP code/Maserati-ARP/Maserati-ARP/Dataset/germancarforum_cleaned_NLP_final_v1.csv", low_memory=False)

In [3]:
test = df["post_text"][:100]
docs = test

# NLP Pineline

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
docs_tokens = []
tmp = []
for doc in tqdm(docs):
    if doc is None or pd.isnull(doc):
        continue

    tmp = [
        token.lemma_
        for token in nlp(doc)
        if (not token.is_stop) & 
        (not token.is_punct) & 
        (token.is_alpha) &
        (token.pos_ == "PROPN") &
        (token.ent_type_ == "ORG")
        
    ]
    
    docs_tokens.append(tmp)

In [ ]:
# save the list of tokens
with open('tkns_docs.pkl', 'wb') as f:
    pickle.dump(docs_tokens, f)

In [263]:
# load tokens
with open('/Users/irene/CloudStation/Courses/ARP/ARP code/Maserati-ARP/Maserati-ARP/Data-analysis/german_brands_tkns.pkl', 'rb') as f:
    docs_tokens_loaded = pickle.load(f)

In [264]:
# find bigrams and trigrams

# get rid of common terms
common_terms = [
    u"of",
    u"with",
    u"without",
    u"and",
    u"or",
    u"the",
    u"a",
    u"not",
    "be",
    u"to",
    u"this",
    u"who",
    u"in",
]
# find phrases
bigram = Phrases( # capturing bi-grams
    docs_tokens_loaded,
    min_count=50,
    threshold=5,
    max_vocab_size=50000,
    common_terms=common_terms,
)
trigram = Phrases( # capturing tri-grams
    bigram[docs_tokens_loaded],
    min_count=50,
    threshold=5,
    max_vocab_size=50000,
    common_terms=common_terms,
)

# uncomment if bi-grammed, tokenized document is preferred

# docs_phrased = [bigram[line] for line in docs_tokens]
docs_phrased = [trigram[bigram[line]] for line in docs_tokens_loaded] # tokenized documents and join 
                                                                # the tokens associated with 
                                                                # bi- or tri-grams.

In [265]:
docs_phrased_list = [item for sublist in docs_phrased for item in sublist]

# Count

In [266]:
brand_counts = dict(Counter(docs_phrased_list))

In [267]:
count = pd.DataFrame(list(brand_counts.items()), columns=['Brand', 'Count'])

In [268]:
count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15213 entries, 0 to 15212
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Brand   15213 non-null  object
 1   Count   15213 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 237.8+ KB


In [269]:
count['Brand'] = count['Brand'].astype(str)
count = count[count['Brand'].str.len() >= 2]
count = count[count['Brand'].str.len() < 25]

In [270]:
# count["Brand"] = count["Brand"].apply(lambda x: x[0])
# count['Brand'] = count['Brand'].astype(str)
count = count.groupby("Brand")["Count"].sum().reset_index()

# Visual

In [271]:
filter = count[count["Count"]>213]

In [275]:
# DTM racing competition
# Brabus Brabus, founded 1977 in Bottrop (Ruhr Area), Germany , is a high-performance aftermarket tuning company which specializes in Mercedes-Benz
# NEDC The New European Driving Cycle
# GM GMC brand
# Glock F1 driver
# BEV Eletric car
# PS horse power
# CDI Common Rail Diesel Injection are comonly used in MB
filter.loc[filter['Brand'].isin(['Benz', 'AMG', "Mercedes", "Daimler", 
                                 "CL", "Class", "SLS", "Maybach", "SL",
                                 "SLK", "CLK", "ML", "GL", "CLA", "Merc",
                                 "GLC", "MB", "GLS", "GLE", "SLC",
                                 "SLR", "GLA", "CLC", "RWD", "CLS", "CDI",
                                 "EQS", "DaimlerChrysler", "MBUSA",
                                 "Gullwing", "EQC", "GLB", "Benzes", "BS",
                                 "Zetsche", "Wagener", "Mercs"]), 'Brand'] = 'Mercedes Benz'
filter.loc[filter['Brand'].isin(['BMWs']), 'Brand'] = 'BMW'
filter.loc[filter['Brand'].isin(['Coupé']), 'Brand'] = 'Coupe'
filter.loc[filter['Brand'].isin(['aston', 'martin', 'am']), 'Brand'] = 'aston martin'
filter.loc[filter['Brand'].isin(['LS']), 'Brand'] = 'Lexus'
filter.loc[filter['Brand'].isin(['Clubman', 'Cooper', 'MINI']), 'Brand'] = 'Mini'
filter.loc[filter['Brand'].isin(['Mclaren']), 'Brand'] = 'McLaren'
filter.loc[filter['Brand'].isin(['Phantom', "Rolls", "Roll royce", "royce"]), 'Brand'] = 'Rolls_Royce'
filter.loc[filter['Brand'].isin(['LWB', "Rolls", "RR"]), 'Brand'] = 'Range_Rover'
filter.loc[filter['Brand'].isin(['VAG', "Golf", "GTI", "VG", "VW"]), 'Brand'] = 'Volkswagen'
filter = filter.groupby('Brand', as_index=False)['Count'].sum()
filter = filter.sort_values(ascending=False, by="Count")

In [276]:
car_brand_names = [
    'Mercedes Benz', 'BMW', 'Ferrari', 'Audi', 'McLaren', 'Renault', 'Porsche', 'Volkswagen', 'Bugatti', 'Ford',
    'Rolls_Royce', 'Lexus', 'Bentley', 'Toyota', 'Range_Rover', 'Honda', 'MINI', 'Ford', 'Maserati', 'Mazda', "Fiat",
    'Lotus', 'Nissan', 'Tesla', 'Chrysler', 'Jaguar', 'Jeep', 'Opel', 'Peugeot', 'Lamborghini', 'Hyundai',
    'Subaru', 'Kia', 'Infiniti', 'Pagani', 'Caterham', 'Dodge', 'Skoda', 'Acura', 'Alfa', 'Cadillac', 'Mini', 'Volvo'
]

# Create a new column and label the brand as "Car Brand" or "Non-Car Brand"
filter['Category'] = filter['Brand'].apply(lambda x: 'Car Brand' if x in car_brand_names else 'Non-Car Brand')

In [277]:
filter_brand = filter[filter["Category"]=="Car Brand"]

In [ ]:
# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(filter_brand.set_index('Brand')['Count'].to_dict())

# Display the generated word cloud
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Car Brand Word Cloud")
plt.show()

In [ ]:
filter_brand_s = filter_brand.sort_values(by='Count', ascending=False)
highlight_colors = {
    'BMW': '#EB96EB',  # Red for BMW
    'Ferrari': "pink",
    'Maserati': "red",
    'Porsche': '#AFB83B',  # Green for Porsche
    'Audi': '#00DBFF',  # Blue for Audi
    'Mercedes Benz': '#FFC100',  # Yellow for Mercedes Benz
    'Land Rover': '#FF00FF',  # Magenta for Land Rover
    'Jaguar': '#6ACCBC',  # Cyan for Jaguar
    'Alfa Romeo': '#800080',  # Purple for Alfa Romeo
    'Aston Martin': '#FFA500',  # Orange for Aston Martin
}

# Create a list to store the marker colors for each brand
marker_colors = [highlight_colors.get(brand, '#002C5F') for brand in filter_brand_s["Brand"]]
fig = go.Figure(data=[
    go.Bar(
        x=filter_brand_s["Brand"],
        y=filter_brand_s["Count"],
        marker_color=marker_colors
    )
])

fig.update_layout(
    title="Brand Frequency",
    xaxis_title="Brands",
    yaxis_title="Count (log)",
    yaxis_type="log"
)

fig.show()

# Co-occurrence of Maserati

In [278]:
# load tokens
with open('/Users/irene/CloudStation/Courses/ARP/ARP code/Maserati-ARP/Maserati-ARP/Data-analysis/BRTopic_tkns_docs.pkl', 'rb') as f:
    df_t = pickle.load(f)

In [279]:
main = [', '.join(item) if item else None for item in df_t]

In [280]:
main_co = pd.DataFrame(main, columns=["text"])

In [281]:
main_co = main_co.dropna()

In [282]:
lowercase_list = [brand.lower() for brand in docs_phrased_list]

In [283]:
def my_function(x):
  return list(dict.fromkeys(x))

lowercase_filter = my_function(lowercase_list)

In [284]:
import re
import pandas as pd
from collections import defaultdict

def find_co_occurrence(df, target_brand, car_brands):
    co_occurrence_counts = defaultdict(int)

    for _, row in df.iterrows():
        text = row['text']
        words = text.split(', ')

        if target_brand in words:
            for brand in car_brands:
                if brand in words and brand != target_brand:
                    co_occurrence_counts[brand] += 1

    return co_occurrence_counts

car_brands = lowercase_filter

# Finding co-occurrence of Maserati with other car brands
co_occurrence_counts = find_co_occurrence(main_co, "maserati", car_brands)

co_occurrence_df = pd.DataFrame(list(co_occurrence_counts.items()), columns=["brands", "co_occurrence_count"])

In [285]:
co_occurrence_df_all = co_occurrence_df.copy()

In [286]:
co_occurrence_df_all = co_occurrence_df_all.groupby("brands")["co_occurrence_count"].sum().reset_index()

In [287]:
co_occurrence_df_all = co_occurrence_df_all.sort_values(by='co_occurrence_count', ascending=False)

In [288]:
co_occurrence_df_all.head(10)

,brands,co_occurrence_count
322,car,141
1257,like,117
1284,look,98
1483,new,93
1373,mercedes,78
779,ferrari,69
916,good,66
577,design,65
685,engine,64
2376,year,64


In [289]:
co_occurrence_df_brandonly = co_occurrence_df_all.copy()

In [290]:
co_occurrence_df_brandonly.loc[co_occurrence_df_brandonly['brands'].isin(['benz', 'amg', "mercedes", "daimler", 
                                 "cl", "class", "sls", "maybach", "sl",
                                 "slk", "clk", "ml", "gl", "cla", "merc",
                                 "glc", "mb", "gls", "gle", "slc",
                                 "slr", "gla", "clc", "rwd", "cls", "cdi",
                                 "eqs", "daimlerchrysler", "mbusa",
                                 "gullwing", "eqc", "glb", "benzes", "bs",
                                 "zetsche", "wagener", "mercs"]), 'brands'] = 'mercedes benz'
co_occurrence_df_brandonly.loc[co_occurrence_df_brandonly['brands'].isin(['bmws']), 'brands'] = 'bmw'
co_occurrence_df_brandonly.loc[co_occurrence_df_brandonly['brands'].isin(['coupé']), 'brands'] = 'coupe'
co_occurrence_df_brandonly.loc[co_occurrence_df_brandonly['brands'].isin(['ls']), 'brands'] = 'lexus'
co_occurrence_df_brandonly.loc[co_occurrence_df_brandonly['brands'].isin(['clubman', 'cooper', 'mini']), 'brands'] = 'mini'
co_occurrence_df_brandonly.loc[co_occurrence_df_brandonly['brands'].isin(['aston', 'martin', 'am']), 'brands'] = 'aston martin'
co_occurrence_df_brandonly.loc[co_occurrence_df_brandonly['brands'].isin(['mclaren']), 'brands'] = 'mcLaren'
co_occurrence_df_brandonly.loc[co_occurrence_df_brandonly['brands'].isin(['alfa',"romeo"]), 'brands'] = 'alfa romeo'
co_occurrence_df_brandonly.loc[co_occurrence_df_brandonly['brands'].isin(['phantom', "rolls", "roll royce", "royce"]), 'brands'] = 'rolls_royce'
co_occurrence_df_brandonly.loc[co_occurrence_df_brandonly['brands'].isin(['lwb']), 'brands'] = 'range_rover'
co_occurrence_df_brandonly.loc[co_occurrence_df_brandonly['brands'].isin(['vag', "golf", "gti", "vg", "vw"]), 'brands'] = 'volkswagen'
co_occurrence_df_brandonly = co_occurrence_df_brandonly.groupby('brands', as_index=False)['co_occurrence_count'].sum()
co_occurrence_df_brandonly = co_occurrence_df_brandonly.sort_values(ascending=False, by="co_occurrence_count")

In [291]:
co_brandonly_filter = co_occurrence_df_brandonly[co_occurrence_df_brandonly["co_occurrence_count"]>20]
co_brandonly_filter = co_brandonly_filter[co_brandonly_filter['brands'].str.len() >= 2]

In [292]:
co_brandonly_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, 1346 to 489
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   brands               103 non-null    object
 1   co_occurrence_count  103 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


In [293]:
co_brandonly_filter

,brands,co_occurrence_count
1346,mercedes benz,334
317,car,141
1234,like,117
1261,look,98
1454,new,93
...,...,...
1645,premium,21
1804,sale,21
1221,level,21
2175,turbo,21


# Visual

In [294]:
highlight_colors = {
    'bmw': '#00DBFF',
    'ferrari': "#00DBFF",
    'maserati': "#00DBFF",
    'porsche': '#00DBFF', 
    'audi': '#00DBFF', 
    'mercedes benz': '#00DBFF',
    'bentley': '#00DBFF',
    'mcLaren': '#00DBFF', 
    'lexus': '#00DBFF', 
    'renault': '#00DBFF',
    'honda':'#00DBFF',
    'chrysler':'#00DBFF',
    'alfa':'#00DBFF',
    'ford':'#00DBFF',
    'toyota':'#00DBFF',
    'aston martin':'#00DBFF',
    'fiat':'#00DBFF',
    'range_rover':'#00DBFF',
    'jaguar':'#00DBFF',
    'mini':'#00DBFF',
    'lotus':'#00DBFF',
    'volvo':'#00DBFF',
    'rolls_royce':'#00DBFF',
    'hyundai':'#00DBFF',
    'lamborghini':'#00DBFF',
    'bugatti':'#00DBFF',
    'mazda':'#00DBFF',
    'kia':'#00DBFF',
    'jeep':'#00DBFF',
    'nissan':'#00DBFF',
    'cadillac':'#00DBFF',
    'infiniti':'#00DBFF',
    'tesla':'#00DBFF',
    'peugeot':'#00DBFF',
    'dodge':'#00DBFF',
    'luxury':'red',
}

marker_colors = [highlight_colors.get(brand, '#002C5F') for brand in co_brandonly_filter["brands"]]
fig = go.Figure(data=[
    go.Bar(
        x=co_brandonly_filter["brands"],
        y=co_brandonly_filter["co_occurrence_count"],
        marker_color=marker_colors
    )
])

fig.update_layout(
    title="Co_occurrence of Maserati with Brands (german)",
    xaxis_title="Brands",
    yaxis_title="Count (log)",
    yaxis_type="log"
)
fig.update_xaxes(dtick=1,tickfont=dict(size=10))
fig.show()

In [312]:
brands_to_filter = ["mercedes benz", "ferrari", "aston martin", "bmw", "porsche", "audi", "bentley", "alfa romeo", "rolls_royce", "jaguar"]
particular = co_brandonly_filter[co_brandonly_filter["brands"].isin(brands_to_filter)]

In [314]:
particular = particular.sort_values(ascending=False, by="co_occurrence_count")

In [315]:
particular

,brands,co_occurrence_count
49,mercedes benz,334
24,ferrari,69
1,aston martin,63
7,bmw,62
0,alfa romeo,61
60,porsche,41
2,audi,40
5,bentley,35
73,rolls_royce,33
35,jaguar,23


In [320]:
particular.to_csv("gp.csv")

In [319]:
marker_colors = "#002C5F"
fig = go.Figure(data=[
    go.Bar(
        x=particular["brands"],
        y=particular["co_occurrence_count"],
        marker_color=marker_colors
    )
])

fig.update_layout(
    title="Co-occurrence of Maserati with Top 10 Car Brands in GermanCarForum",
    xaxis_title="Car Brand",
    yaxis_title="Count",
    plot_bgcolor='rgba(0,0,0,0)' 
)
fig.update_xaxes(dtick=1,tickfont=dict(size=10))
fig.show()